# Create a graph

Imports

In [1]:
import networkx as nx
import random
import powerlaw
import os
import scipy.sparse
from scipy.sparse.linalg import eigs
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from matplotlib.image import imread

Parameters of the network

In [145]:
# Parameters
num_nodes = 1000
probability = 0.01  # Probability of edge creation

Create the graph

In [146]:
graph_name = "ER_graph"

In [147]:
# Generate an Erdős–Rényi graph
G = nx.erdos_renyi_graph(n=num_nodes, p=probability)
#G = nx.powerlaw_cluster_graph(n=num_nodes, m=5, p=0.3)
# Assign random weights to the edges, rounded to 3 decimals
for u, v in G.edges():
    G[u][v]['weight'] = round(random.uniform(0.1, 10.0), 3) 

In [ ]:
bet = nx.edge_betweenness_centrality(G,weight="weight", backend="parallel")

Save the network as edgelist for reproducibility of results

In [149]:
output_dir = "./networks/"
output_file = os.path.join(output_dir, f"{graph_name}.edgelist")
os.makedirs(output_dir, exist_ok=True)

nx.write_weighted_edgelist(G, f"./networks/{graph_name}")

# Read the graph

Read the graph from the file

In [2]:
syn_name = "ER_graph"
input_file = f"./networks/{syn_name}"

# Read the weighted graph
syn = nx.read_weighted_edgelist(input_file)

Function to read txt graphs

In [2]:
def load_weighted_graph(file_path):
    G = nx.Graph()  
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:  
            parts = line.split()
            if len(parts) == 3:
                node1 = int(parts[0])
                node2 = int(parts[1])
                weight = float(parts[2])
                G.add_edge(node1, node2, weight=weight)
    return G

# file paths
file_path_karate = 'networks/karate.txt'
file_path_kangaroos = 'networks/Kangaroos.txt'

karate = load_weighted_graph(file_path_karate)
kangaroos = load_weighted_graph(file_path_kangaroos)
# names 
karate_name = "karate_graph"
kangaroos_name = "kangaroos_graph"

Check they are properly read

In [ ]:
# Print some edges with weights to verify
print("Sample edges from the synthetic graph:")
for u, v, data in list(syn.edges(data=True))[:10]:
    print(f"({u}, {v}) - weight: {data['weight']}")

In [ ]:
# Print some edges with weights to verify
print("Sample edges from the karate graph:")
for u, v, data in list(karate.edges(data=True))[:10]:
    print(f"({u}, {v}) - weight: {data['weight']}")

In [ ]:
# Print some edges with weights to verify
print("Sample edges from the kangaroos graph:")
for u, v, data in list(kangaroos.edges(data=True))[:10]:
    print(f"({u}, {v}) - weight: {data['weight']}")

# Start

## Utils

### Explore the graph

In [74]:
def save_graph_properties(graph, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Basic properties
    num_nodes = graph.number_of_nodes()
    num_edges = graph.number_of_edges()
    edge_weights = [d['weight'] for _, _, d in graph.edges(data=True)]
    min_weight = min(edge_weights)
    max_weight = max(edge_weights)
    avg_weight = sum(edge_weights) / len(edge_weights)
    
    # Graph density
    density = nx.density(graph)
    
    # Degree distribution
    degrees = [d for _, d in graph.degree()]
    weighted_degrees = [sum(graph[u][v]['weight'] for v in graph.neighbors(u)) for u in graph.nodes()]
    
    # Connected components
    connected_components = list(nx.connected_components(graph))
    num_components = len(connected_components)
    component_sizes = [len(c) for c in connected_components]
    
    # Clustering coefficient
    clustering_coeff = nx.average_clustering(graph, weight='weight')
    
    # Additional properties
    # Average path length (weighted)
    avg_path_length = nx.average_shortest_path_length(graph, weight='weight') if nx.is_connected(graph) else None
    
    # Diameter (weighted)
    diameter = nx.diameter(graph, weight='weight') if nx.is_connected(graph) else None
    
    # Radius (weighted)
    radius = nx.radius(graph, weight='weight') if nx.is_connected(graph) else None
    
    # Degree assortativity
    degree_assortativity = nx.degree_assortativity_coefficient(graph)
    
    # Edge connectivity
    edge_connectivity = nx.edge_connectivity(graph)
    
    # Compute the largest eigenvalue of the adjacency matrix
    adj_matrix = nx.to_numpy_array(graph, weight='weight')
    eigenvalues = np.linalg.eigvals(adj_matrix)
    first_eigenvalue = max(eigenvalues)  # Get the largest eigenvalue
    
    # Prepare the results for saving to file
    results = [
        f"Number of nodes: {num_nodes}",
        f"Number of edges: {num_edges}",
        f"Minimum edge weight: {min_weight}",
        f"Maximum edge weight: {max_weight}",
        f"Average edge weight: {avg_weight:.2f}",
        f"Graph density: {density:.4f}",
        f"Number of connected components: {num_components}",
        f"Sizes of connected components: {component_sizes}",
        f"Global clustering coefficient: {clustering_coeff:.4f}",
        f"Average path length: {avg_path_length}",
        f"Diameter: {diameter}",
        f"Radius: {radius}",
        f"Degree assortativity: {degree_assortativity:.4f}",
        f"Edge connectivity: {edge_connectivity}",
        f"First eigenvalue: {first_eigenvalue:.6f}"  # Add the first eigenvalue
    ]
    
    # Save the results to a text file
    results_file = os.path.join(output_dir, "graph_properties.txt")
    with open(results_file, 'w') as f:
        f.write("\n".join(results))
    
    # Display the contents of the text file
    with open(results_file, 'r') as f:
        print(f.read())
        
    # Plot edge weight distribution
    plt.figure(figsize=(10, 6))
    plt.hist(edge_weights, bins=20, color='skyblue', edgecolor='black')
    plt.title('Edge Weight Distribution', fontsize=14)
    plt.xlabel('Weight', fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "edge_weight_distribution.png"))
    plt.close()

    # Plot degree distribution
    plt.figure(figsize=(10, 6))
    plt.hist(degrees, bins=20, color='orange', edgecolor='black')
    plt.title('Degree Distribution', fontsize=14)
    plt.xlabel('Degree', fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "degree_distribution.png"))
    plt.close()

    # Plot weighted degree distribution
    plt.figure(figsize=(10, 6))
    plt.hist(weighted_degrees, bins=20, color='green', edgecolor='black')
    plt.title('Weighted Degree Distribution', fontsize=14)
    plt.xlabel('Weighted Degree', fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "weighted_degree_distribution.png"))
    plt.close()


### Experiments

Get the largest eigenvalue from a given adjacency matrix

In [12]:
def get_largest_eigenval(
    adj_matrix: scipy.sparse._csr.csr_array
) -> np.float64:
    eigenvalues, _ = eigs(adj_matrix, k=1, which='LM')  # 'LM' for largest magnitude eigenvalue
    # The largest eigenvalue is the real part of the first eigenvalue
    largest_eigenvalue = np.real(eigenvalues[0])
    return largest_eigenvalue

In [13]:
def sqrt_trace_of_adj_squared(
    adj_matrix: scipy.sparse._csr.csr_array
) -> np.float64:
    # Ensure the matrix is square
    if adj_matrix.shape[0] != adj_matrix.shape[1]:
        raise ValueError("The adjacency matrix must be square.")
    
    # Compute adj_matrix^2
    adj_squared = adj_matrix @ adj_matrix  # Efficient sparse matrix multiplication
    
    # Compute the trace of adj_matrix^2
    trace = adj_squared.diagonal().sum()  # Sum of diagonal elements
    
    # Compute the square root of the trace
    result = np.sqrt(trace)
    
    return result

Get the amount of change (psi) by using spectral norm ||A||_2 = λ_1

In [14]:
def get_amount_of_change(
    perturbed_adj_matrix: scipy.sparse._csr.csr_array, 
    adj_matrix: scipy.sparse._csr.csr_array
) -> np.float64:
    perturbation_matrix = perturbed_adj_matrix - adj_matrix 
    return sqrt_trace_of_adj_squared(perturbation_matrix)/sqrt_trace_of_adj_squared(adj_matrix)

#### Define centrality metrics

In [50]:
def degree_centrality(
    adj_matrix: scipy.sparse._csr.csr_array
) -> dict:
    adj_matrix_dense = adj_matrix.todense()
    # Convert the dense adjacency matrix to a NetworkX graph
    G_from_adj = nx.from_numpy_array(np.array(adj_matrix_dense))
    # Compute degree centrality using networkx
    degree_centrality = {node: degree for node, degree in G_from_adj.degree(weight='weight')}
    return degree_centrality

def weighted_katz_centrality(
    adj_matrix: scipy.sparse._csr.csr_array
) -> dict:
    adj_matrix_dense = adj_matrix.todense()
    # Convert the dense adjacency matrix to a NetworkX graph
    G_from_adj = nx.from_numpy_array(np.array(adj_matrix_dense))
    # get the alpha value 
    max_alpha = 1/get_largest_eigenval(adj_matrix)
    alpha = max_alpha * 0.9 #we reduce max alpha by 10%
    # Compute degree centrality using networkx
    katz_centrality = nx.katz_centrality(G_from_adj, alpha=alpha, weight = "weight")
    return katz_centrality

def weighted_eigenvector_centrality(
    adj_matrix: scipy.sparse._csr.csr_array
) -> dict:
    adj_matrix_dense = adj_matrix.todense()
    # Convert the dense adjacency matrix to a NetworkX graph
    G_from_adj = nx.from_numpy_array(np.array(adj_matrix_dense))
    # Compute degree centrality using networkx
    eigenvector_centrality = nx.eigenvector_centrality(G_from_adj, weight = "weight")
    return eigenvector_centrality

Wrap the computation of different centrality metrics in a single function

In [16]:
def get_centrality_scores(
    adj_matrix: scipy.sparse._csr.csr_array, 
    func
) -> dict:
    return func(adj_matrix)

Get the deformation of centrality metrics (ζ)

In [17]:
def get_deformation_centrality_metric(
    perturbed_adj_matrix: scipy.sparse._csr.csr_array, 
    adj_matrix: scipy.sparse._csr.csr_array, 
    func
) -> np.float64:
    centrality_metric_adj_matrix = get_centrality_scores(adj_matrix, func)
    centrality_metric_perturbed_adj_matrix  = get_centrality_scores(perturbed_adj_matrix, func)
    diff_centrality_metric_dict = {
        key: centrality_metric_perturbed_adj_matrix[key] - centrality_metric_adj_matrix[key] 
        for key in centrality_metric_perturbed_adj_matrix.keys()
    }
    euclidean_norm_diff = np.sqrt(sum(value**2 for value in diff_centrality_metric_dict.values()))
    euclidean_norm = np.sqrt(sum(value**2 for value in centrality_metric_adj_matrix.values()))
    return euclidean_norm_diff/euclidean_norm

#### Define perturbation methods

In [18]:
def perturb_based_on_edge_betweenness(
    G_sparse: nx.classes.graph.Graph,
    E_prime: dict,
    edge_betweenness: dict,
    consider_only_E_prime: bool = False,
    scaling: str='Min-Max'
) -> scipy.sparse._csr.csr_array:
    G_new = G_sparse.copy()
    E_prime_edges = list(E_prime.keys())
    filtered_edges = {edge: edge_betweenness.get(edge, 0) for edge in E_prime_edges} if consider_only_E_prime else edge_betweenness
    if scaling == 'Min-Max':
        min_edge_betweenness = min(filtered_edges.values())
        max_edge_betweenness = max(filtered_edges.values())
        if max_edge_betweenness > min_edge_betweenness:
            normalized_centrality = np.array([
                (filtered_edges[edge] - min_edge_betweenness) / (max_edge_betweenness - min_edge_betweenness)
                for edge in E_prime_edges
            ]) 
    else:
        total_betweenness = sum(filtered_edges.values())
        normalized_centrality = np.array([filtered_edges[edge] / total_betweenness for edge in E_prime_edges])
    random_values = np.random.rand(len(E_prime_edges))
    mask = random_values < normalized_centrality
    edges_to_remove = np.array(E_prime_edges)[mask]
    G_new.remove_edges_from(edges_to_remove)

    return nx.adjacency_matrix(G_new)


def perturb_based_on_weight(
    G_sparse: nx.classes.graph.Graph,
    E_prime: dict,
    edge_weights = dict,
    consider_only_E_prime: bool = False,
    scaling: str = 'Min-Max'
) -> scipy.sparse._csr.csr_array:
    G_new = G_sparse.copy()
    E_prime_edges = list(E_prime.keys())
    filtered_edges = {edge: E_prime[edge] for edge in E_prime_edges} if consider_only_E_prime else edge_weights
    if scaling == 'Min-Max':
        min_weight = min(filtered_edges.values())
        max_weight = max(filtered_edges.values())
        if max_weight > min_weight:
            normalized_weight = np.array([
                (filtered_edges[edge] - min_weight) / (max_weight - min_weight)
                for edge in E_prime_edges
            ])
    else:
        total_weight = sum(filtered_edges.values())
        normalized_weight = np.array([filtered_edges[edge] / total_weight for edge in E_prime_edges])
    random_values = np.random.rand(len(E_prime_edges))
    mask = random_values < normalized_weight
    edges_to_remove = np.array(E_prime_edges)[mask]
    G_new.remove_edges_from(edges_to_remove)

    return nx.adjacency_matrix(G_new)


Wrap the methods to perturb a matrix in a single function

In [19]:
def perturb_adj_matrix(
    G_sparse: nx.classes.graph.Graph,
    E_prime: dict,
    edge_betweenness: dict = None,
    edge_weights: dict = None,
    func: str = 'betweenness',
    scaling: str = 'Min-Max',
    consider_only_E_prime: bool = False
):
    if func == 'betweenness':
        if edge_betweenness is None:
            raise ValueError("edge_betweenness must be provided when func is 'betweenness'")
        return perturb_based_on_edge_betweenness(
            G_sparse, E_prime, edge_betweenness, consider_only_E_prime, scaling
        )
    elif func == 'weight':
        if edge_weights is None:
            raise ValueError("edge_weights must be provided when func is 'weight'")
        return perturb_based_on_weight(
            G_sparse, E_prime, edge_weights, consider_only_E_prime, scaling
        )

#### Subset E'

Get the subset of edges that can fail

In [20]:
def get_edges_subset(G_loaded, fraction: float) -> dict:
    # Get all edges with weights from the graph and store in a dictionary
    edges_with_weights = {(u, v): data['weight'] for u, v, data in G_loaded.edges(data=True)}
    # Calculate the number of edges to select based on the fraction
    num_edges_to_select = int(len(edges_with_weights) * fraction)
    # Randomly sample
    random_edges = random.sample(list(edges_with_weights.items()), num_edges_to_select)
    random_edges_dict = dict(random_edges)
    
    return random_edges_dict


### Plots

In [21]:
def plot_results(
    x_list: list, 
    y_list: list, 
    x_label: str, 
    y_label: str, 
    title: str,
    plot_linear_regression: bool = False, 
    save_path: str = None
):
    x_array = np.array(x_list)
    y_array = np.array(y_list)
    X = x_array.reshape(-1, 1)
    y = y_array

    model = LinearRegression()
    model.fit(X, y)
    slope = model.coef_[0]
    intercept = model.intercept_
    x_range = np.linspace(min(x_array), max(x_array), 500).reshape(-1, 1)
    y_pred = model.predict(x_range)

    plt.figure(figsize=(8, 6))
    plt.scatter(x_array, y_array, color='blue', alpha=0.7, label='Data Points')
    if plot_linear_regression:
        plt.plot(x_range, y_pred, color='red', label='Regression Line', linewidth=2)
        plt.text(
            0.05, 0.95,
            fr'$y = {slope:.3e}x + {intercept:.3e}$',
            fontsize=12,
            transform=plt.gca().transAxes,
            verticalalignment='top'
        )
    plt.title(fr'{title}', fontsize=16)
    plt.xlabel(fr'{x_label}', fontsize=14)
    plt.ylabel(fr'{y_label}', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()

    if save_path:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.show()
        plt.close()
    else:
        plt.show()

def display_proportional_grid(directory: str):
    plot_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(('.png', '.jpg', '.jpeg'))]
    
    if not plot_files:
        print(f"No plots found in the directory: {directory}")
        return
    
    # Determine grid dimensions
    num_plots = len(plot_files)
    rows = 1 if num_plots <= 3 else int(num_plots**0.5)  # Ensure at least one row
    cols = (num_plots + rows - 1) // rows  # Ensure all plots fit in the grid

    # Create a grid of plots
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4))
    axes = axes.flatten()

    for ax, plot_file in zip(axes, plot_files):
        img = imread(plot_file)
        ax.imshow(img)
        ax.axis('off')  # Remove axis

    # Turn off any unused axes
    for ax in axes[len(plot_files):]:
        ax.axis('off')

    plt.tight_layout()
    plt.show()


### all plots toghter

In [99]:


def plot_results(
    x_list: list,  # x is the same for all datasets
    y_lists: list,  # List of y data lists
    labels: list,
    x_label: str,
    y_label: str,
    title: str,
    plot_linear_regression: bool = False,
    save_path: str = None
):
    x_array = np.array(x_list)
    X = x_array.reshape(-1, 1)
    
    colors = ['blue', 'green', 'orange']  # Define a set of colors
    plt.figure(figsize=(8, 6))
    
    for i, (y_list, label) in enumerate(zip(y_lists, labels)):
        y_array = np.array(y_list)
        
        plt.plot(
            x_array, y_array, alpha=0.7, label=label, color=colors[i % len(colors)],
            marker='o', linestyle='-', linewidth=1.5  # Marker for points and line style
        )
        
        if plot_linear_regression:
            # Linear regression
            model = LinearRegression()
            model.fit(X, y_array)
            slope = model.coef_[0]
            intercept = model.intercept_
            x_range = np.linspace(min(x_array), max(x_array), 500).reshape(-1, 1)
            y_pred = model.predict(x_range)
            plt.plot(
                x_range, y_pred,
                label=f'Regression {i+1}: y = {slope:.3e}x + {intercept:.3e}',
                color=colors[i % len(colors)],
                linewidth=2,
                linestyle='--'
            )
    plt.title(title, fontsize=16)
    plt.xlabel(x_label, fontsize=14)
    plt.ylabel(y_label, fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()

    if save_path:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.show()
        plt.close()
    else:
        plt.show()

def display_proportional_grid(directory: str):
    # Collect all image files in the directory
    plot_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(('.png', '.jpg', '.jpeg'))]
    
    if not plot_files:
        print(f"No plots found in the directory: {directory}")
        return
    
    # Determine grid dimensions
    num_plots = len(plot_files)
    rows = int(num_plots**0.5)  # Number of rows (square root for proportional grid)
    cols = (num_plots + rows - 1) // rows  # Adjust number of columns to fit all images

    # Create a grid of plots
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4))
    axes = axes.flatten()

    for ax, plot_file in zip(axes, plot_files):
        img = imread(plot_file)
        ax.imshow(img)
        ax.axis('off')  # Remove axis for clean display

    # Turn off any unused axes
    for ax in axes[len(plot_files):]:
        ax.axis('off')

    plt.tight_layout()
    plt.show()


# Set configuration

# Explore the graph

synthetic graph

In [ ]:
output_dir_syn = f"properties/{syn_name}"
save_graph_properties(syn, output_dir_syn)
display_proportional_grid(output_dir_syn)

karate graph

In [ ]:
output_dir_karate = f"properties/{karate_name}"
save_graph_properties(karate, output_dir_karate)
display_proportional_grid(output_dir_karate)

kangaroos graph

In [ ]:
output_dir_kangaroos = f"properties/{kangaroos_name}"
save_graph_properties(kangaroos, output_dir_kangaroos)
display_proportional_grid(output_dir_kangaroos)

# Init experiments

### Define adj matrix, edge_betweenness, weights

In [92]:
#synthetic
adj_matrix_syn = nx.adjacency_matrix(syn)
edge_betweenness_syn = nx.edge_betweenness_centrality(syn, weight="weight")#, backend = "parallel")
edge_weights_syn = {(u, v): data['weight'] for u, v, data in syn.edges(data=True)}
#enron
adj_matrix_karate = nx.adjacency_matrix(karate)
edge_betweenness_karate = nx.edge_betweenness_centrality(karate, weight="weight")#, backend = "parallel")
edge_weights_karate = {(u, v): data['weight'] for u, v, data in karate.edges(data=True)}
#kangaroos
adj_matrix_kangaroos = nx.adjacency_matrix(kangaroos)
edge_betweenness_kangaroos = nx.edge_betweenness_centrality(kangaroos, weight="weight")#, backend = "parallel")
edge_weights_kangaroos = {(u, v): data['weight'] for u, v, data in kangaroos.edges(data=True)}
# tau list for all
tau_list = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
# number of iteration for all
n_iter = 10

Run the experiment for different taus and graphs

### Synthetic graph

In [ ]:
func = "betweenness"
scaling = "Min-Max"

if func == "betweenness":
    edge_betweenness_dict = edge_betweenness_syn
    edge_weights_dict = None
elif func == "weight":
    edge_betweenness_dict = None
    edge_weights_dict = edge_weights_syn
    

psi_list_syn = []
zeta_degree_centrality_list_syn = []
zeta_katz_centrality_list_syn = []
zeta_eigenvector_centrality_list_syn = []

for tau in tau_list:
    print(f"Processing tau: {tau}")
    psi_cumm = 0
    zeta_degree_centrality_cumm = 0
    zeta_closeness_centrality_cumm = 0
    zeta_katz_centrality_cumm = 0
    zeta_eigenvector_centrality_cumm = 0
    for _ in range(n_iter):
        E_prime = get_edges_subset(syn, tau)

        '''Type of perturbation: func = 'betweenness' or func= 'weight'
           Object to be pass w.r.t type of perturbation: edge_weights=edge_weights else edge_betweenness = edge_betweenness (just one the other set to None as default)
           Type of Normalization: scaling = 'Min-Max' or scaling = 'Sum'
           condider only the subset for removal: consider_only_E_prime = True or false
        '''
        perturbed_adj_matrix = perturb_adj_matrix(
            syn, 
            E_prime, 
            edge_weights = edge_weights_dict,
            edge_betweenness = edge_betweenness_dict, 
            func = func, 
            scaling = scaling, 
            consider_only_E_prime = False
        ) 
        psi_cumm += get_amount_of_change(perturbed_adj_matrix, adj_matrix_syn)
        zeta_degree_centrality_cumm += get_deformation_centrality_metric(perturbed_adj_matrix, adj_matrix_syn, degree_centrality) 
        zeta_katz_centrality_cumm += get_deformation_centrality_metric(perturbed_adj_matrix, adj_matrix_syn, weighted_katz_centrality) 
        zeta_eigenvector_centrality_cumm += get_deformation_centrality_metric(perturbed_adj_matrix, adj_matrix_syn, weighted_eigenvector_centrality) 
        
    
    mean_psi = psi_cumm/n_iter
    mean_zeta_degree_centrality = zeta_degree_centrality_cumm/n_iter
    mean_zeta_katz_centrality = zeta_katz_centrality_cumm/n_iter
    mean_zeta_eigenvector_centrality = zeta_eigenvector_centrality_cumm/n_iter

    psi_list_syn.append(mean_psi)
    zeta_degree_centrality_list_syn.append(mean_zeta_degree_centrality)
    zeta_katz_centrality_list_syn.append(mean_zeta_katz_centrality)
    zeta_eigenvector_centrality_list_syn.append(mean_zeta_eigenvector_centrality)

### Karate graph

In [ ]:
func = "betweenness"
scaling = "Min-Max"

if func == "betweenness":
    edge_betweenness_dict = edge_betweenness_karate
    edge_weights_dict = None
elif func == "weight":
    edge_betweenness_dict = None
    edge_weights_dict = edge_weights_karate
    

psi_list_karate = []
zeta_degree_centrality_list_karate = []
zeta_katz_centrality_list_karate = []
zeta_eigenvector_centrality_list_karate = []

for tau in tau_list:
    print(f"Processing tau: {tau}")
    psi_cumm = 0
    zeta_degree_centrality_cumm = 0
    zeta_closeness_centrality_cumm = 0
    zeta_katz_centrality_cumm = 0
    zeta_eigenvector_centrality_cumm = 0
    for _ in range(n_iter):
        E_prime = get_edges_subset(karate, tau)

        '''Type of perturbation: func = 'betweenness' or func= 'weight'
           Object to be pass w.r.t type of perturbation: edge_weights=edge_weights else edge_betweenness = edge_betweenness (just one the other set to None as default)
           Type of Normalization: scaling = 'Min-Max' or scaling = 'Sum'
           condider only the subset for removal: consider_only_E_prime = True or false
        '''
        perturbed_adj_matrix = perturb_adj_matrix(
            karate, 
            E_prime, 
            edge_weights = edge_weights_dict,
            edge_betweenness = edge_betweenness_dict, 
            func = func, 
            scaling = scaling, 
            consider_only_E_prime = False
        ) 
        psi_cumm += get_amount_of_change(perturbed_adj_matrix, adj_matrix_karate)
        zeta_degree_centrality_cumm += get_deformation_centrality_metric(perturbed_adj_matrix, adj_matrix_karate, degree_centrality) 
        zeta_katz_centrality_cumm += get_deformation_centrality_metric(perturbed_adj_matrix, adj_matrix_karate, weighted_katz_centrality) 
        zeta_eigenvector_centrality_cumm += get_deformation_centrality_metric(perturbed_adj_matrix, adj_matrix_karate, weighted_eigenvector_centrality) 
        
    
    mean_psi = psi_cumm/n_iter
    mean_zeta_degree_centrality = zeta_degree_centrality_cumm/n_iter
    mean_zeta_katz_centrality = zeta_katz_centrality_cumm/n_iter
    mean_zeta_eigenvector_centrality = zeta_eigenvector_centrality_cumm/n_iter

    psi_list_karate.append(mean_psi)
    zeta_degree_centrality_list_karate.append(mean_zeta_degree_centrality)
    zeta_katz_centrality_list_karate.append(mean_zeta_katz_centrality)
    zeta_eigenvector_centrality_list_karate.append(mean_zeta_eigenvector_centrality)

### Kangaroos graph

In [ ]:
func = "betweenness"
scaling = "Min-Max"

if func == "betweenness":
    edge_betweenness_dict = edge_betweenness_kangaroos
    edge_weights_dict = None
elif func == "weight":
    edge_betweenness_dict = None
    edge_weights_dict = edge_weights_kangaroos
    

psi_list_kangaroos = []
zeta_degree_centrality_list_kangaroos = []
#zeta_closeness_centrality_list_kangaroos = []
zeta_katz_centrality_list_kangaroos = []
zeta_eigenvector_centrality_list_kangaroos = []

for tau in tau_list:
    print(f"Processing tau: {tau}")
    psi_cumm = 0
    zeta_degree_centrality_cumm = 0
    zeta_closeness_centrality_cumm = 0
    zeta_katz_centrality_cumm = 0
    zeta_eigenvector_centrality_cumm = 0
    for _ in range(n_iter):
        E_prime = get_edges_subset(kangaroos, tau)

        '''Type of perturbation: func = 'betweenness' or func= 'weight'
           Object to be pass w.r.t type of perturbation: edge_weights=edge_weights else edge_betweenness = edge_betweenness (just one the other set to None as default)
           Type of Normalization: scaling = 'Min-Max' or scaling = 'Sum'
           condider only the subset for removal: consider_only_E_prime = True or false
        '''
        perturbed_adj_matrix = perturb_adj_matrix(
            kangaroos, 
            E_prime, 
            edge_weights = edge_weights_dict,
            edge_betweenness = edge_betweenness_dict, 
            func = func, 
            scaling = scaling, 
            consider_only_E_prime = False
        ) 
        psi_cumm += get_amount_of_change(perturbed_adj_matrix, adj_matrix_kangaroos)
        zeta_degree_centrality_cumm += get_deformation_centrality_metric(perturbed_adj_matrix, adj_matrix_kangaroos, degree_centrality) 
        zeta_katz_centrality_cumm += get_deformation_centrality_metric(perturbed_adj_matrix, adj_matrix_kangaroos, weighted_katz_centrality) 
        zeta_eigenvector_centrality_cumm += get_deformation_centrality_metric(perturbed_adj_matrix, adj_matrix_kangaroos, weighted_eigenvector_centrality) 
        
    
    mean_psi = psi_cumm/n_iter
    mean_zeta_degree_centrality = zeta_degree_centrality_cumm/n_iter
    mean_zeta_katz_centrality = zeta_katz_centrality_cumm/n_iter
    mean_zeta_eigenvector_centrality = zeta_eigenvector_centrality_cumm/n_iter

    psi_list_kangaroos.append(mean_psi)
    zeta_degree_centrality_list_kangaroos.append(mean_zeta_degree_centrality)
    zeta_katz_centrality_list_kangaroos.append(mean_zeta_katz_centrality)
    zeta_eigenvector_centrality_list_kangaroos.append(mean_zeta_eigenvector_centrality)

# Plots

In [29]:
save_path = f"plots/{func.lower()}/{scaling.lower()}/"

## Plot the amount of change (psi) vs. percentage of edges that can fail (tau)

In [ ]:
psi_lists = [psi_list_syn,psi_list_karate,psi_list_kangaroos]
plot_results(
    x_list = tau_list,
    y_lists = psi_lists,
    labels = ["Synthetic","Karate","Kangaroos"],
    x_label = r"$\tau$",
    y_label = r"$\psi$",
    title = r"Amount of change vs. percentage of edges that can fail",
    plot_linear_regression = False,
    save_path = save_path + f"tau_vs_psi_niter={n_iter}"
)

## Plot the deformation of different centrality metrics (zeta) vs. percentage of edges that can fail (tau)

### Degree Centrality

In [ ]:
zeta_degree_centrality_lists = [zeta_degree_centrality_list_syn,zeta_degree_centrality_list_karate,zeta_degree_centrality_list_kangaroos]
plot_results(
    x_list = tau_list,
    y_lists = zeta_degree_centrality_lists,
    labels = ["Synthetic","Karate","Kangaroos"],
    x_label = r"$\tau$",
    y_label = r"$\zeta$",
    title = r"Deformation of degree centrality vs. percentage of edges that can fail",
    plot_linear_regression = False,
    save_path =  save_path + f"tau_vs_zeta_degree_centrality_niter={n_iter}"
)

### Katz Centrality

In [ ]:
zeta_katz_centrality_lists = [zeta_katz_centrality_list_syn,zeta_katz_centrality_list_karate,zeta_katz_centrality_list_kangaroos]
plot_results(
    x_list = tau_list,
    y_lists = zeta_katz_centrality_lists,
    labels = ["Synthetic","Karate","Kangaroos"],
    x_label = r"$\tau$",
    y_label = r"$\zeta$",
    title = r"Deformation of Katz centrality vs. percentage of edges that can fail",
    plot_linear_regression = False,
    save_path =  save_path + f"tau_vs_zeta_katz_centrality_niter={n_iter}"
)

### Eigenvector Centrality

In [ ]:
zeta_eigenvector_centrality_lists = [zeta_eigenvector_centrality_list_syn,zeta_eigenvector_centrality_list_karate,zeta_eigenvector_centrality_list_kangaroos]
plot_results(
    x_list = tau_list,
    y_lists = zeta_eigenvector_centrality_lists,
    labels = ["Synthetic","Karate","Kangaroos"],
    x_label = r"$\tau$",
    y_label = r"$\zeta$",
    title = r"Deformation of Eigenvector centrality vs. percentage of edges that can fail",
    plot_linear_regression = False,
    save_path =  save_path + f"tau_vs_zeta_eigenvector_centrality_niter={n_iter}"
)

# Display all plots

In [ ]:
display_proportional_grid(save_path)